In [3]:
import jax
jax.__version__

'0.4.30'

## DeviceArray properties

In [4]:
import numpy as np
import jax.numpy as jnp

In [5]:
np.array([1, 42, 31337])

array([    1,    42, 31337])

In [6]:
jnp.array([1, 42, 31337])

Array([    1,    42, 31337], dtype=int32)

In [7]:
np.sum([1, 42, 31337])

31380

In [8]:
try:
  jnp.sum([1, 42, 31337])
except TypeError as e:
  print(e)

sum requires ndarray or scalar arguments, got <class 'list'> at position 0.


In [9]:
jnp.sum(jnp.array([1, 42, 31337]))

Array(31380, dtype=int32)

In [10]:
arr = jnp.array([1, 42, 31337])

In [11]:
arr.ndim

1

In [12]:
arr.shape

(3,)

In [13]:
arr.dtype

dtype('int32')

In [14]:
arr.size

3

In [15]:
arr.nbytes

12

## Devices

In [16]:
import jax

In [17]:
jax.devices()

[cuda(id=0)]

In [18]:
jax.devices('cpu')

[CpuDevice(id=0)]

In [19]:
jax.device_count('gpu')

1

In [20]:
jax.local_devices()

[cuda(id=0)]

In [21]:
arr = jnp.array([1, 42, 31337])

In [23]:
arr.devices()

{cuda(id=0)}

In [24]:
arr_cpu = jax.device_put(arr, jax.devices('cpu')[0])

In [26]:
arr_cpu.devices()

{CpuDevice(id=0)}

In [27]:
arr.devices()

{cuda(id=0)}

In [28]:
arr_host = jax.device_get(arr)

In [29]:
type(arr_host)

numpy.ndarray

In [30]:
arr_host

array([    1,    42, 31337], dtype=int32)

In [39]:
arr2 = arr + arr_cpu

In [40]:
arr2.devices()

{CpuDevice(id=0)}

In [32]:
arr_gpu = jax.device_put(arr, jax.devices('gpu')[0])

In [34]:
arr_gpu.devices()

{cuda(id=0)}

In [37]:
try:
  arr_gpu + arr_cpu
except ValueError as e:
  print(e)

Received incompatible devices for jitted computation. Got argument x1 of jax.numpy.add with shape int32[3] and device ids [0] on platform GPU and argument x2 of jax.numpy.add with shape int32[3] and device ids [0] on platform CPU


## Asyncronous dispatch

In [41]:
import jax

In [51]:
a = jnp.array(range(1000000)).reshape((1000,1000))

In [52]:
a.shape

(1000, 1000)

In [53]:
a.devices()

{cuda(id=0)}

In [72]:
%time x = jnp.dot(a,a)

CPU times: user 656 μs, sys: 0 ns, total: 656 μs
Wall time: 404 μs


In [73]:
%time x = jnp.dot(a,a).block_until_ready()

CPU times: user 886 μs, sys: 719 μs, total: 1.61 ms
Wall time: 1.7 ms


In [74]:
%time x = np.asarray(jnp.dot(a,a))

CPU times: user 870 μs, sys: 706 μs, total: 1.58 ms
Wall time: 2.14 ms


In [75]:
a_cpu = jax.device_put(a, jax.devices('cpu')[0])

In [76]:
a_cpu.devices()

{CpuDevice(id=0)}

In [77]:
%time x = jnp.dot(a_cpu,a_cpu).block_until_ready()

CPU times: user 111 ms, sys: 0 ns, total: 111 ms
Wall time: 9.27 ms


## GPU diagnostics

In [78]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


/home/jet/.pyenv/versions/3.11.0/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [79]:
#!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [80]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Aug  4 20:59:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.27                 Driver Version: 560.70         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0  On |                  N/A |
|  0%   40C    P8              8W /  285W |   15787MiB /  16376MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Immutability

In [81]:
import numpy as np
import jax.numpy as jnp

In [82]:
a_jnp = jnp.array(range(10))
a_np  = np.array(range(10))

In [83]:
a_jnp

Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [84]:
a_np[5], a_jnp[5]

(5, Array(5, dtype=int32))

In [85]:
a_np[5] = 100

In [86]:
a_np[5]

100

In [87]:
try:
  a_jnp[5] = 100
except TypeError as e:
  print(e)

'<class 'jaxlib.xla_extension.ArrayImpl'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html


In [88]:
a_jnp = a_jnp.at[5].set(100)

In [89]:
a_jnp[5]

Array(100, dtype=int32)

In [90]:
a_jnp[42]

Array(9, dtype=int32)

In [91]:
a_jnp.at[42].get()

Array(9, dtype=int32)

In [92]:
a_jnp.at[42].get(mode='clip')

Array(9, dtype=int32)

In [93]:
a_jnp.at[42].get(mode='drop')

Array(-2147483648, dtype=int32)

In [94]:
a_jnp.at[42].get(mode='fill', fill_value=-1)

Array(-1, dtype=int32)

In [95]:
a_jnp = a_jnp.at[42].set(100)
a_jnp

Array([  0,   1,   2,   3,   4, 100,   6,   7,   8,   9], dtype=int32)

In [96]:
a_jnp = a_jnp.at[42].set(100, mode='clip')
a_jnp

Array([  0,   1,   2,   3,   4, 100,   6,   7,   8, 100], dtype=int32)

## Working with float64

In [98]:
# this only works on startup!
import jax
jax.config.update("jax_enable_x64", True)

In [99]:
import jax
import jax.numpy as jnp

In [100]:
# this will not work on TPU backend. Try using CPU or GPU.
x = jnp.array(range(10), dtype=jnp.float64)
x.dtype

dtype('float64')

In [102]:
x.devices()

{cuda(id=0)}

In [103]:
xc = jax.device_put(x, jax.devices('cpu')[0])

In [104]:
xc.devices()

{CpuDevice(id=0)}

In [105]:
xc.dtype

dtype('float64')

In [106]:
xb16 = jnp.array(range(10), dtype=jnp.bfloat16)
xb16.dtype

dtype(bfloat16)

In [107]:
xb16.nbytes

20

In [108]:
x16 = jnp.array(range(10), dtype=jnp.float16)
x16.dtype

dtype('float16')

In [109]:
x16.nbytes

20

In [110]:
xb16+x16

Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)

In [111]:
xb16+xb16

Array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18], dtype=bfloat16)

## jax.numpy & jax.lax

In [112]:
jax.config.update("jax_enable_x64", False)

In [113]:
import jax.numpy as jnp
from jax import lax
from jax import random

In [114]:
jnp.add(42, 42.0)

Array(84., dtype=float32, weak_type=True)

In [115]:
jnp.add(42.0, 42.0)

Array(84., dtype=float32, weak_type=True)

In [116]:
try:
   lax.add(42, 42.0)
except TypeError as e:
  print(e)

ValueError: Cannot lower jaxpr with verifier errors:
	op requires the same element type for all operands and results
		at loc("jit(add)/jit(main)/add"(callsite("<module>"("/tmp/ipykernel_37180/1506868229.py":2:3) at callsite("InteractiveShell.run_code"("/home/jet/.local/share/virtualenvs/JAX-in-Action-2DrGcPDg/lib/python3.11/site-packages/IPython/core/interactiveshell.py":3577:20) at callsite("InteractiveShell.run_ast_nodes"("/home/jet/.local/share/virtualenvs/JAX-in-Action-2DrGcPDg/lib/python3.11/site-packages/IPython/core/interactiveshell.py":3517:19) at callsite("InteractiveShell.run_cell_async"("/home/jet/.local/share/virtualenvs/JAX-in-Action-2DrGcPDg/lib/python3.11/site-packages/IPython/core/interactiveshell.py":3334:29) at callsite("_pseudo_sync_runner"("/home/jet/.local/share/virtualenvs/JAX-in-Action-2DrGcPDg/lib/python3.11/site-packages/IPython/core/async_helpers.py":128:8) at callsite("InteractiveShell._run_cell"("/home/jet/.local/share/virtualenvs/JAX-in-Action-2DrGcPDg/lib

In [117]:
lax.add(jnp.float32(42), 42.0)

Array(84., dtype=float32)

In [118]:
def random_augmentation(image, augmentations, rng_key):
   '''A function that applies a random transformation to an image'''
   augmentation_index = random.randint(key=rng_key, minval=0, maxval=len(augmentations), shape=())
   augmented_image = lax.switch(augmentation_index, augmentations, image)
   return augmented_image

In [119]:
add_noise_func = lambda x: x+10
horizontal_flip_func = lambda x: x+1
rotate_func = lambda x: x+2
adjust_colors_func = lambda x: x+3

augmentations = [
   add_noise_func,
   horizontal_flip_func,
   rotate_func,
   adjust_colors_func
]


In [120]:
image = jnp.array(range(100))

In [121]:
random_augmentation(image, augmentations, random.PRNGKey(211))

Array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int32)